<a href="https://colab.research.google.com/github/Rahama04/Rahama04/blob/main/Copy_of_Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

import tensorflow_datasets as tfds

import tensorflow as tf
tfds.disable_progress_bar()



In [ ]:
dataset, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)

train_dataset, test_dataset = dataset['train'], dataset['test']


In [ ]:
for example, label in train_dataset.take(1):
  print("text: ", example.numpy())
  print("label: ", label.numpy())



text:  b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
label:  0


In [ ]:
BUFFER_SIZE=10000
BATCH_SIZE=64


In [ ]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [ ]:
for example, label in train_dataset.take(1):
   print("texts: ", example.numpy()[:3])
   print()
   print("label: ", label.numpy() [:3])


texts:  [b"Dirty Sanchez is the more extreme, British version of Jackass in which the four boys (Pritchard, Dainton, Joycey and Pancho) go to great lengths to hurt and humiliate each other. The reason this show is better than Jackass is because most of the stunts are not planned which makes the reaction much more funny. There are 3 series of the show, the first follows them around and takes a long look at their lives eg. there's an episode on their love lives,jobs etc. The second series sends the boys to try out different occupations. The third follows their European tour. It seems that the boys get more and more daring as the show progresses through the series. In my opinion the third series is the best, but trust me when i say, if you have a week stomach DO NOT WATCH, as you are lightly to see a fair amount of blood and puke in every episode."
 b"Like a lot of series pilots, Dark Angel's opener shows a mixture of great potential and a slight problem finding its own feet. Not that thi

In [ ]:
experimental.preprocessing.TextVectorization layer

SyntaxError: ignored

In [ ]:
VOCAB_SIZE = 1000

encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=VOCAB_SIZE)

encoder.adapt(train_dataset.map(lambda text, label:text))

In [ ]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]


array(['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i',
       'this', 'that', 'br', 'was', 'as', 'for', 'with', 'movie', 'but'],
      dtype='<U7')

In [ ]:
encoded_example = encoder(example)[:3].numpy()
encoded_example

array([[ 1,  1,  7, ...,  0,  0,  0],
       [39,  4,  1, ...,  0,  0,  0],
       [ 1, 37,  1, ...,  0,  0,  0]])

In [ ]:
model = tf.keras.Sequential([encoder, tf.keras.layers.Embedding(input_dim=len(encoder.get_vocabulary()), output_dim=64, mask_zero=True), 
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)), tf.keras.layers.Dense(64, activation='relu'),
                             tf.keras.layers.Dense(1)])


In [ ]:
sample_text=(" the movie was cool" "animations were good")

predictions=model.predict(np.array([sample_text]))

print(predictions[0])


[-0.00098234]


In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), optimizer=tf.keras.optimizers.Adam(1e-4), metrics=["accuracy"])


In [ ]:
model.fit(train_dataset, epochs=10, validation_data=test_dataset, validation_steps=30)

Epoch 1/10
391/391 [==============================] - 717s 2s/step - loss: 0.6846 - accuracy: 0.5134 - val_loss: 0.6402 - val_accuracy: 0.5500
Epoch 2/10
347/391 [=========================>....] - ETA: 1:16 - loss: 0.5867 - accuracy: 0.6658

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)

print("test_loss",test_loss
      
      print("test accuracy:",test_acc))